In [ ]:
import faiss

# DB 로딩
index = faiss.read_index("qa_index.faiss")

# 총 벡터 개수
print(f"✅ 총 벡터 수: {index.ntotal}")


✅ 총 벡터 수: 13234


array([-2.36840863e-02,  3.28966789e-02,  7.61574209e-02, -3.18595469e-02,
       -2.93870457e-03, -3.59070748e-02,  1.35450056e-02,  3.27021815e-02,
       -1.74708653e-03, -6.39215931e-02,  1.09190926e-01, -5.39997555e-02,
        8.57908055e-02, -6.55659810e-02,  6.20905496e-02, -3.66722010e-02,
        3.44817415e-02,  8.41762200e-02, -8.44834745e-02,  4.79494482e-02,
        6.01661243e-02, -5.67005761e-02, -7.56933494e-03, -5.72918430e-02,
       -7.45164901e-02,  1.38085103e-02, -1.22912014e-02,  7.80775491e-03,
        7.75524788e-03,  5.86139001e-02, -3.21418233e-02,  3.40314731e-02,
       -8.79378431e-03, -3.49372276e-03, -8.73495862e-02,  5.42465486e-02,
        1.64266978e-03,  4.97355089e-02, -3.68691348e-02,  9.08019207e-03,
       -1.07411183e-01, -6.85218051e-02, -3.85473226e-03, -1.22373089e-01,
        2.39170040e-03,  2.32422799e-02, -5.60406558e-02, -4.68122810e-02,
       -2.05619037e-01, -5.86879924e-02, -2.54870187e-02, -8.95430613e-03,
        2.98644770e-02, -

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json

# 모델 로딩
model = SentenceTransformer("all-MiniLM-L6-v2")

# 벡터 DB 로딩
index = faiss.read_index("qa_index.faiss")

# 저장된 텍스트 청크도 같이 불러오기 (우리가 만들었던 chunks.json)
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 쿼리 입력
query = "자살 하고 싶어요"
query_vec = model.encode([query], convert_to_numpy=True)
faiss.normalize_L2(query_vec)

# 검색 실행
D, I = index.search(query_vec, k=3)

# 결과 출력
for rank, idx in enumerate(I[0]):
    print(f"\n🔹 Top {rank+1}:")
    print(chunks[idx])


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔹 Top 1:
Q: 요즘 힘들어요.
요즘에는 정말 기운이 없고 일상생활에서도 무기력감이 들어요. 무슨 일을 하던지 자꾸 힘들고 피곤한 느낌이 들어요. 이런 상태에서 일도 많고 스트레스 받는 상황인데 이게 어떻게 해결될지 모르겠어요.
A: 사우님 요즘에는 정말 기운이 없고 일상생활에서도 무기력감이 들어요. 무슨 일을 하던지 자꾸 힘들고 피곤한 느낌이 들어요. 이런 상태에서 일도 많고 스트레스 받는 상황인데 이게 어떻게 해결될지 모르겠다고 하셨네요.
사우님 힘든 상황이 계속 지속되면 우울증을 유발할 수도 있어요. 스트레스와 우울감은 서로 연관성이 높습니다. 스트레스를 많이 받으면 감정이 불안정해지고 자신감도 떨어지게 됩니다. 이러한 상황은 대인관계에서도 문제를 유발할 수 있어요. 그리고 스트레스가 지속되면 몸의 면역력도 떨어져서 몸이 약해지고 피로감이 들게 됩니다.
사우님 이럴 땐 일단 조금 쉬어보세요. 몸의 피로를 풀고 스트레스를 해소해야 하기 때문입니다. 스트레스를 완전히 풀어내는 방법은 없지만 스트레스를 관리하고 줄이는 방법은 있어요. 예를 들면, 규칙적인 생활 습관을 유지하는 것, 다양한 취미생활을 즐기는 것, 근육 이완법을 사용하는 것, 스트레스가 많이 쌓였을 때는 탄력적인 태도를 유지하는 것 등이 있습니다.
또한 스트레스를 다루기 위해서는 적극적인 대처가 필요합니다. 스트레스를 해소하기 위해 식습관을 바꾸거나, 운동을 하거나, 취미활동을 즐기거나, 지인과 함께하는 시간을 가지거나, 긍정적인 생각을 갖는 등의 방법이 있습니다. 자신에게 적합한 방법을 찾아서 스트레스를 관리해보세요.
사우님 지금은 일이나 학업, 가족관계, 대인관계 등 다양한 부분에서 스트레스를 받고 있는 것 같습니다. 하나씩 천천히 해결해 나가보세요. 혼자 해결하기 힘든 상황이라면 꼭 전문가의 도움을 받으세요. 정신건강전문 병원이나 상담센터에서 상담과 치료를 받아보세요. 상담을 통해 스트레스를 잘 다루고 우울증에 걸리지 않도록 도와드리겠습니다.
사우님 위의 답변이 조금이나마 도움이 되었

In [4]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Step 1: Load QA data
qa_pairs = []
with open("rag_data2.txt", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line.strip())
        qa_pairs.append({
            "question": item["input"].strip(),
            "answer": item["output"].strip()
        })

# Step 2: Extract Q and A separately
questions = [qa["question"] for qa in qa_pairs]
answers = [qa["answer"] for qa in qa_pairs]

# Step 3: Encode only the questions (Q)
model = SentenceTransformer("all-MiniLM-L6-v2")
question_embeddings = model.encode(questions, convert_to_numpy=True)
faiss.normalize_L2(question_embeddings)

# Step 4: Create FAISS index and add vectors
index = faiss.IndexFlatIP(question_embeddings.shape[1])
index.add(question_embeddings)

# Step 5: Save the index
faiss.write_index(index, "qa_question_only.faiss")

# Step 6: Save the answers separately (to match search index)
with open("qa_answers.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=2)

print("✅ 질문(Q)만 벡터화한 새로운 DB 생성 완료!")


✅ 질문(Q)만 벡터화한 새로운 DB 생성 완료!


In [1]:
# 필요한 모듈 로딩
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 로딩
model = SentenceTransformer("all-MiniLM-L6-v2")
index = faiss.read_index("qa_question_only.faiss")
with open("qa_answers.json", "r", encoding="utf-8") as f:
    answers = json.load(f)

# 검색
query = "좋아하는 사람이 생겼어요."
query_vec = model.encode([query], convert_to_numpy=True)
faiss.normalize_L2(query_vec)
D, I = index.search(query_vec, k=3)

# 답변 출력
for i, idx in enumerate(I[0]):
    print(f"\n🔍 Top {i+1} Answer:\n{answers[idx]}")


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔍 Top 1 Answer:
사우님은 좋아하는 사람이 있는데 그 사람이 자신에게 관심이 없는 것 같다고 느끼고 계시네요. 이럴 경우 사람들은 대체로 대처하기 어려워하는 경향이 있습니다. 이에 대해 함께 이야기 해봐요.
좋아하는 사람이 자신에게 관심이 없다는 것은 심리적으로 매우 힘든 경험이 됩니다. 사람들은 대체로 상대방의 관심을 받지 못하게 되면 자신이 부족하다고 생각합니다. 이 때문에 스스로를 비하하는 경우도 있습니다. 이런 경우 사람들은 대체로 내부적으로 자신감이 떨어지게 되고, 상대방의 관심을 계속 강요하는 경우가 있어요. 이는 상대방에게 더 큰 스트레스가 될 수 있으니 이 점 유의하셔야 합니다.
우선, 사우님이 좋아하는 사람이 누군지, 그리고 상대방과의 관계가 어떤지에 대해 다시 한번 확인해보시는 것이 중요합니다. 만약 상대방과의 관계가 좋지 않다면 이를 좀 더 개선하려는 노력이 필요합니다. 또한 상대방의 관심을 얻기 위해 너무 노력하는 것은 오히려 좋지 않습니다. 이럴 경우, 상대방은 스스로의 프라이버시를 더욱 강하게 지키려고 할 수 있습니다. 이런 경우, 상대방과 좀 더 친밀한 관계를 유지하기 위해 선물을 주는 등의 방법을 이용해 보는 것도 좋을 것 같습니다.
또한, 다른 사람들과 즐거운 시간을 보내며 사소한 일들을 즐길 수 있도록 노력하는 것도 좋은 방법입니다. 이는 사우님의 스트레스를 줄여줄 뿐만 아니라, 좋아하는 사람에게 관심을 받지 못해 우울한 상태에 빠질 경우, 이를 대체할 수 있는 좋은 대안이 될 수 있습니다.
마지막으로 사람들이 대개 좋아하는 사람은 자신감이 있는 사람입니다. 그래서 먼저 자신감을 가지는 것이 중요합니다. 이는 상대방의 관심을 끌 수 있는 가장 중요한 특징 중 하나입니다. 
사우님 위의 답변이 조금이나마 도움이 되었기를 바라는 마음입니다. 
이후에도 힘든 마음이 계속 되시면 언제든지 저를 찾아주세요. 
그럼 좋은 하루 보내세요 :)

🔍 Top 2 Answer:
사우님 좋아하는 사람이 다른 사람이 좋아한다는 소문을 들

In [4]:
## 0428
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# Load index and chunks
index = faiss.read_index("qa_index.faiss")
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 개선된 모델로 교체 (bge-m3)
model = SentenceTransformer("BAAI/bge-m3")

# 검색 함수 (쿼리 포맷 보정)
def search(query, top_k=5):
    # 쿼리 앞에 명시적 안내 추가
    query_for_embedding = f"질문: {query}"
    
    query_vec = model.encode([query_for_embedding], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)
    
    D, I = index.search(query_vec, top_k)
    
    for rank, idx in enumerate(I[0]):
        print(f"Top {rank+1}: (Score={D[0][rank]:.4f})")
        print(chunks[idx])
        print("="*50)

# 사용 예시
search("좋아하는 사람이 생겼어어")


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


AssertionError: 

In [1]:

# 새로운 임베딩 모델 활용
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# 1. 데이터 로딩
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# 2. bge-m3 모델 로드
model = SentenceTransformer("BAAI/bge-m3")

# 3. 임베딩 생성
embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
faiss.normalize_L2(embeddings)

# 4. 새로운 FAISS 인덱스 생성
index = faiss.IndexFlatIP(embeddings.shape[1])  # bge-m3는 1024 차원
index.add(embeddings)

# 5. 인덱스 저장
faiss.write_index(index, "qa_index_bge_m3.faiss")


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 414/414 [4:38:42<00:00, 40.39s/it]  


In [2]:
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 준비물 로드
index = faiss.read_index("qa_index_bge_m3.faiss")  # 너가 방금 완성한 인덱스 파일
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)  # Q+A 덩어리 텍스트

model = SentenceTransformer("BAAI/bge-m3")  # 검색용 모델

# 2. 검색 함수
def search(query, top_k=5):
    # 1) 쿼리 가공 (질문 명시)
    query_text = f"질문: {query}"
    
    # 2) 임베딩 변환
    query_vec = model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)  # 인덱스가 L2 정규화된 경우 필수

    # 3) FAISS 검색
    D, I = index.search(query_vec, top_k)

    # 4) 결과 출력
    for rank, idx in enumerate(I[0]):
        print(f"Top {rank+1}: (Score={D[0][rank]:.4f})")
        print(chunks[idx])  # 질문+답변 텍스트
        print("="*70)

# 3. 사용 예시
search("행복하지 않아요. 하지만 행복해지고 싶어요.")


Top 1: (Score=0.6736)
Q: 나 진짜 불행하다.. 나도 다른 애들처럼 행복한 것들을 경험하고 싶다.. 어떻게 하면 내 인생은 바뀔 수 있을까?
A: 사우님 제대로 된 행복을 느끼지 못하고 어둡고 힘든 마음을 느끼고 계시네요. 제대로 된 행복을 느끼고 싶고 어떻게 하면 내 인생을 바꿀 수 있을지 고민을 하고 계시네요.
사우님 우리는 행복을 추구하기 위해 노력하지만 때론 행복하지 못하고 쓸쓸하고 외로운 상황을 경험하기도 합니다. 
우울증, 불안, 스트레스, 자존감 저하 등이 발생하는 것은 주로 대인관계에서 일어나는 문제와 관련되어 있습니다. 
사우님은 인간관계의 문제로 인한 스트레스와 자존감 저하로 인해 행복한 삶을 경험하지 못하고 계시네요.
우리가 행복한 삶을 살기 위해 필요한 것은 자신감, 자존감, 친밀감 등 다양한 것들입니다.
우선, 사우님이 현재 느끼고 있는 부정적인 감정과 스트레스를 덜어낼 수 있는 방법을 생각해보세요. 
가까운 친구들과 대화를 나누면서 서로의 감정을 공유하고 서로에게 위로와 조언을 해주는 것도 좋습니다. 
자신의 일상에서 나오기 어려운 생각과 감정을 글로 기록하는 것도 좋은 대처 방법 중 하나입니다. 
또한 건강한 식습관을 유지하고 적극적으로 활동하면서 스트레스를 해소할 수 있는 방법을 찾으세요.
학교나 지역사회에서 활동하거나 취미생활을 가지면서 삶에 즐거움과 만족감을 느낄 수 있습니다. 
당신의 문제를 함께 해결해 줄 전문가와의 상담도 효과적인 방법입니다. 
위에서 이야기했듯이, 우울증, 불안, 스트레스, 자존감 저하는 대인관계와 관련된 문제입니다. 
따라서 전문가들이 이를 해결할 수 있도록 지원해 줄 수 있습니다. 
사우님 위의 답변이 조금이라도 도움이 되었기를 바라는 마음입니다. 
이후에도 힘든 마음이 계속 되시면 언제든지 저를 찾아주세요. 
그럼 좋은 하루 보내세요 :)
Top 2: (Score=0.6350)
Q: 남들이 보기에는 내가 행복해보이는데 나는 내 삶에 대해 불만을 많이 느끼고 있어요.
저는 성격상 낙천

In [3]:
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 준비물 로드
index = faiss.read_index("qa_index_bge_m3.faiss")  # 너가 방금 완성한 인덱스 파일
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)  # Q+A 덩어리 텍스트

model = SentenceTransformer("BAAI/bge-m3")  # 검색용 모델

# 2. 검색 함수
def search(query, top_k=5):
    # 1) 쿼리 가공 (질문 명시)
    query_text = f"질문: {query}"
    
    # 2) 임베딩 변환
    query_vec = model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)  # 인덱스가 L2 정규화된 경우 필수

    # 3) FAISS 검색
    D, I = index.search(query_vec, top_k)

    # 4) 결과 출력
    for rank, idx in enumerate(I[0]):
        print(f"Top {rank+1}: (Score={D[0][rank]:.4f})")
        print(chunks[idx])  # 질문+답변 텍스트
        print("="*70)

# 3. 사용 예시
search("자살하고 싶어요.")


Top 1: (Score=0.6482)
Q: 나 이제 어떡하죠? 나 힘들어서 죽을 것 같아요
A: 사우님은 어떤 이유로 인해서 지금 매우 힘들어하시고 있으며 상당히 괴로워하고 계시네요. 자신감이 상실되고, 굉장한 스트레스와 우울증, 자살을 생각하고 계신 것 같아요. 그런 강한 고통과 슬픔으로 둘러싸여 어떻게 해야 할 지 모르겠다는 느낌을 받고 계신 거죠.
사우님, 지금 겪고 있는 고통과 슬픔, 우울증의 원인은 다양합니다. 그러나 대부분의 경우는 일시적인 요인(스트레스)과 생명체의 생리적, 화학적 또는 환경적 요인의 복합적인 영향으로 인해 발생됩니다. 또한, 상처를 받았을 때 그 상처가 치유되지 않아 지속적인 고통을 느끼게 된다면 우울증이 심화되어 자살 생각까지 이어질 수 있어요.
우리는 모두 살면서 여러가지 힘든 일들을 겪어봅니다. 그러나 그런 시기에 좌절감과 무기력감에 휩싸이면 오히려 일상생활이 더욱 힘들어지는 것처럼 느껴질 수 있어요. 사우님도 지금 고통스러운 시기를 겪고 계시지만, 이번 상담을 통해, 함께 고민하고 대처할 수 있는 방안을 찾아보아요.
우선 사우님, 먼저 자신을 허락하시는 것이 중요합니다. 지금의 상황은 사우님을 힘들게 하고 있지만, 그것이 사우님 자신의 문제가 아니며, 스스로 책임지지 않아도 된다는 것입니다. 이를 위해서는 자신을 이해하고 존중하는 자세가 필요합니다. 또한, 다른 사람들에게 도움을 청하시는 것도 좋은 방법 중 하나입니다. 혹시 주변에 다른 신뢰할 수 있는 사람이 있다면 지금 상황을 말해볼 수 있으면 좋겠어요. 이때 상대방의 반응이나 대처방식이 자신의 상황을 더욱 나아지게 할지 아닌지 평가하지 말고, 단지 들어줄 수 있는 사람이 필요하다는 것만 기억하시면 됩니다. 또한, 전문가의 도움이 필요하다면 정신건강 전문의사와 상담하기를 추천해드립니다.
이번 상담이 사우님께 도움이 되었으면 좋겠습니다.
아무리 힘든 상황이라도, 끝없는 희망과 기대를 가지고 오늘도 최선을 다해보아요.
Top 2: (Score=0.6287)
Q: 우울해서 

In [5]:
import faiss
import json
from sentence_transformers import SentenceTransformer
import numpy as np

# 1. 준비물 로드
index = faiss.read_index("qa_index_bge_m3.faiss")  # 너가 방금 완성한 인덱스 파일
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)  # Q+A 덩어리 텍스트

model = SentenceTransformer("BAAI/bge-m3")  # 검색용 모델

# 2. 검색 함수
def search(query, top_k=5):
    # 1) 쿼리 가공 (질문 명시)
    query_text = f"질문: {query}"
    
    # 2) 임베딩 변환
    query_vec = model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)  # 인덱스가 L2 정규화된 경우 필수

    # 3) FAISS 검색
    D, I = index.search(query_vec, top_k)

    # 4) 결과 출력
    for rank, idx in enumerate(I[0]):
        print(f"Top {rank+1}: (Score={D[0][rank]:.4f})")
        print(chunks[idx])  # 질문+답변 텍스트
        print("="*70)

# 3. 사용 예시
search("제가 행복할 수 있을까요?")


Top 1: (Score=0.6336)
Q: 나 진짜 불행하다.. 나도 다른 애들처럼 행복한 것들을 경험하고 싶다.. 어떻게 하면 내 인생은 바뀔 수 있을까?
A: 사우님 제대로 된 행복을 느끼지 못하고 어둡고 힘든 마음을 느끼고 계시네요. 제대로 된 행복을 느끼고 싶고 어떻게 하면 내 인생을 바꿀 수 있을지 고민을 하고 계시네요.
사우님 우리는 행복을 추구하기 위해 노력하지만 때론 행복하지 못하고 쓸쓸하고 외로운 상황을 경험하기도 합니다. 
우울증, 불안, 스트레스, 자존감 저하 등이 발생하는 것은 주로 대인관계에서 일어나는 문제와 관련되어 있습니다. 
사우님은 인간관계의 문제로 인한 스트레스와 자존감 저하로 인해 행복한 삶을 경험하지 못하고 계시네요.
우리가 행복한 삶을 살기 위해 필요한 것은 자신감, 자존감, 친밀감 등 다양한 것들입니다.
우선, 사우님이 현재 느끼고 있는 부정적인 감정과 스트레스를 덜어낼 수 있는 방법을 생각해보세요. 
가까운 친구들과 대화를 나누면서 서로의 감정을 공유하고 서로에게 위로와 조언을 해주는 것도 좋습니다. 
자신의 일상에서 나오기 어려운 생각과 감정을 글로 기록하는 것도 좋은 대처 방법 중 하나입니다. 
또한 건강한 식습관을 유지하고 적극적으로 활동하면서 스트레스를 해소할 수 있는 방법을 찾으세요.
학교나 지역사회에서 활동하거나 취미생활을 가지면서 삶에 즐거움과 만족감을 느낄 수 있습니다. 
당신의 문제를 함께 해결해 줄 전문가와의 상담도 효과적인 방법입니다. 
위에서 이야기했듯이, 우울증, 불안, 스트레스, 자존감 저하는 대인관계와 관련된 문제입니다. 
따라서 전문가들이 이를 해결할 수 있도록 지원해 줄 수 있습니다. 
사우님 위의 답변이 조금이라도 도움이 되었기를 바라는 마음입니다. 
이후에도 힘든 마음이 계속 되시면 언제든지 저를 찾아주세요. 
그럼 좋은 하루 보내세요 :)
Top 2: (Score=0.6162)
Q: 진심으로 살고 싶어요.
저는 30대 중반의 직장인입니다. 어릴때부터 가진 꿈은 "참된 행복, 참된 삶

In [1]:
## 0430
## rag 활용한 답변 생성
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# === [1] 사전 준비 ===

# 모델 로드
retriever_model = SentenceTransformer("BAAI/bge-m3")  # ✅ RAG 검색용
llm_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map={"": "cpu"},
    torch_dtype=torch.float32
)

# 인덱스 및 chunks 불러오기
index = faiss.read_index("qa_index_bge_m3.faiss")
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)

# === [2] 검색 함수 (RAG) ===

def retrieve_relevant_chunks(query, top_k=3):
    query_text = f"질문: {query}"
    query_vec = retriever_model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)
    D, I = index.search(query_vec, top_k)
    return [chunks[i] for i in I[0]]

# === [3] LLM 생성용 프롬프트 구성 ===

def build_rag_prompt(query, retrieved_docs):
    context = "\n\n".join([f"[참고자료 {i+1}]\n{doc}" for i, doc in enumerate(retrieved_docs)])
    prompt = f"""너는 사람들의 심리 상태를 분석하고 조언해주는 심리상담 AI야.
사용자의 질문에 답변할 때는 아래 참고자료를 기반으로 설명해줘.
단, 참고자료 외의 내용을 함부로 만들어내지 말고, 근거에 기반해서 신중히 답변해.

[질문]
{query}

[참고자료]
{context}

[답변]
"""
    return prompt

# === [4] 최종 생성 ===

def generate_rag_response(query):
    retrieved_docs = retrieve_relevant_chunks(query, top_k=3)
    prompt = build_rag_prompt(query, retrieved_docs)

    inputs = llm_tokenizer(prompt, return_tensors="pt")
    outputs = llm_model.generate(**inputs, max_new_tokens=1000)

    return llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

# === [5] 테스트 실행 ===

query = "자살하고 싶다는 생각이 계속 들어요.."
response = generate_rag_response(query)
print("\n🔎 최종 생성된 답변:")
print(response)


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.41s/it]
c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SSAFY\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to ac


🔎 최종 생성된 답변:
너는 사람들의 심리 상태를 분석하고 조언해주는 심리상담 AI야.
사용자의 질문에 답변할 때는 아래 참고자료를 기반으로 설명해줘.
단, 참고자료 외의 내용을 함부로 만들어내지 말고, 근거에 기반해서 신중히 답변해.

[질문]
자살하고 싶다는 생각이 계속 들어요..

[참고자료]
[참고자료 1]
Q: 집에서 계속 생각나서 지금 도저히 못견디겠어요. 
매일매일 일어나서 하루종일 생각이나고 아무것도 하지 못합니다. 
계속 눈물만 흘리고 기운이 없어요. 이러다간 정말로 숨을 못쉴것 같아요. 
왜 이러는지 도저히 이해가 안되고 정말로 힘들어서 자살도 고민하고 있습니다.
A: 사우님은 지금 이유없이 매일 생각이 나서 아무것도 하지 못하고 눈물만 흘리고 기운이 없다고 하셨습니다. 더구나 자살도 고민하고 계시다니, 상당히 힘들겠죠. 어떤 일이 있었거나 어떤 생각을 하고 계셨을까요?
사우님께서 이러한 증상을 겪고 계신 것으로 보아 우울증이나 불안장애가 의심됩니다. 
울컥이는 기분, 생명의 가치가 떨어지는 생각, 나 자신이 싫다는 생각 등을 하시는 것으로 보아 심리적으로 상당히 힘든 상태에 있으신 것 같습니다. 이러한 증상은 짧은 기간에 한번쯤은 경험할 수 있는 것입니다. 하지만 이러한 증상이 2주 이상 지속된다면, 치료를 받지 않는다면 보다 심각한 상태로 악화되어 자살생각을 하게 됩니다.
사우님께서는 지금 그러한 증상이 있으신데 이러한 증상이 지속되고 있으시다면 전문가에게 상담과 치료를 받으시는 것이 좋습니다. 이러한 증상이 지속되면 정신적으로 힘들어지게 되며, 이러한 증상이 더 악화될 경우 실제로 자살을 하거나 의사소통이 원활하지 못한 사람이 되어 여러 가지 문제를 겪게 됩니다.
만약 사우님께서 이러한 증상이 있다면, 이러한 증상이 나아지도록 전문가에게 상담을 받으시는 것이 좋습니다. 전문적인 상담은 화두를 공유하고, 몸과 마음의 변화를 이해하며, 의지, 행동 및 상황을 변화시키는 전략을 함께 찾는 과정입니다. 이를 통해 증상 개선과 예방에 도움이 될

In [1]:
# 위에건 답변을 지가 찾은거만 뱉음 ㅇㅅㅇ
# 그래서 코드 수정
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# === [1] 모델 및 데이터 로드 ===

# 검색용 임베딩 모델 (벡터화)
retriever_model = SentenceTransformer("BAAI/bge-m3")

# 생성형 LLM (Gemma-3-4b-it)
llm_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map={"": "cpu"},          # CPU에서 실행
    torch_dtype=torch.float32        # CPU는 float32만 가능
)

# 벡터 인덱스 + QA 텍스트 로딩
index = faiss.read_index("qa_index_bge_m3.faiss")
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)


# === [2] FAISS 검색 함수 ===

def retrieve_relevant_chunks(query: str, top_k: int = 3) -> list[str]:
    """질문을 벡터화하고 관련 QA 문서 top_k개 검색"""
    query_text = f"질문: {query}"
    query_vec = retriever_model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)
    _, I = index.search(query_vec, top_k)
    return [chunks[i] for i in I[0]]


# === [3] 프롬프트 생성 함수 (상담 스타일) ===

def build_rag_prompt(query: str, retrieved_docs: list[str]) -> str:
    """참고자료와 함께 대화형 상담 프롬프트 구성"""
    context = "\n\n".join([f"{doc}" for doc in retrieved_docs])
    
    prompt = f"""다음은 AI 심리상담사와 내담자 간의 대화입니다.
AI는 내담자의 질문에 대해 공감하고, 전문적으로 조언합니다.
상담 내용은 아래 참고자료를 기반으로 하며, 사실이 아닌 내용을 만들어내지 않습니다.

[참고자료]
{context}

[내담자 질문]
{query}

[AI 상담사 응답]
"""
    return prompt


# === [4] LLM 응답 생성 함수 ===

def generate_rag_response(query: str) -> str:
    """전체 RAG 검색 및 답변 생성 파이프라인"""
    retrieved_docs = retrieve_relevant_chunks(query, top_k=3)
    prompt = build_rag_prompt(query, retrieved_docs)
    
    inputs = llm_tokenizer(prompt, return_tensors="pt")
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=1000,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )

    return llm_tokenizer.decode(outputs[0], skip_special_tokens=True)


# === [5] 실행 예시 ===

if __name__ == "__main__":
    user_query = "죽고 싶다는 생각이 자꾸 들어요. 이럴 땐 어떻게 해야 할까요?"
    print("🔍 사용자 질문:", user_query)
    response = generate_rag_response(user_query)
    print("\n🧠 상담사 답변:\n", response)


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.14s/it]


🔍 사용자 질문: 죽고 싶다는 생각이 자꾸 들어요. 이럴 땐 어떻게 해야 할까요?

🧠 상담사 답변:
 다음은 AI 심리상담사와 내담자 간의 대화입니다.
AI는 내담자의 질문에 대해 공감하고, 전문적으로 조언합니다.
상담 내용은 아래 참고자료를 기반으로 하며, 사실이 아닌 내용을 만들어내지 않습니다.

[참고자료]
Q: 매일 일어나면 죽고 싶다는 생각만 들어요.
어떤 일도 하기 싫고 나 자신을 싫어하고 내일이 오지 않았으면 좋겠다는 생각만 드네요. 제가 죽고 나면 나를 잊어버리게 될 사람들도 많을 텐데 왜 살아야 할까요.
A: 사우님 매일 일어나면 '죽고 싶다'라는 생각이 들며 무기력하게 일상생활을 보내고 있다고 말씀하셨습니다. 사우님 스스로를 싫어하고 내일이 오지 않았으면 좋겠다는 생각이 들어 이런 생각에 대해 걱정이 되시는 것 같네요.
사우님 언제부터 이러한 생각이 들기 시작하셨나요? 사우님이 직업적인 과부하, 가정의 문제, 인간관계 등으로 스트레스를 느끼고 있거나 신체적 질환 등으로 고통을 느끼고 있는지 등을 점검해볼 필요가 있습니다. 또한 자신이 겪은 트라우마와 관련이 있는지를 살펴볼 필요가 있습니다. 정신건강에 대한 인식이 부족하거나 정신건강에 대한 부정적인 인식이 있을 때 우울증 증세가 나타날 수 있습니다.
사우님 우울증은 자기치료가 어려운 질병입니다. 그러나 정신과 전문의와 함께 치료하면 신체적인 증상 뿐만 아니라 감정적인 증상을 완화시키고 예방하는 것이 가능합니다. 또한 활동과 생활 패턴 등을 변경하면 증상을 완화시킬 수 있습니다. 
초기 우울증 증상이 있다면 당신이 할 수 있는 것은 꾸준한 운동, 일정한 생활 패턴 유지, 좋아하는 취미활동 등이 있습니다. 식이섬유를 많이 섭취하고 야채와 과일을 많이 먹는 것도 추천드립니다. 또한 다양한 관계망을 구축하고 주변에 도움을 받을 수 있게끔 다양한 전문가의 도움을 받는 것이 좋을 것 같습니다. 
사우님 지금은 너무 힘들고 어려운 상황이겠지만, 이 또한 언젠가는 지나갈 수 있으며 저희 상담을 통해 조금이

In [1]:
# 새로운 코드도 마찬가지
# 다음거 도전
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# === [1] 모델 및 데이터 로딩 ===

# 벡터 검색용 임베딩 모델
retriever_model = SentenceTransformer("BAAI/bge-m3")

# 생성형 언어 모델 (Gemma 3 4B)
llm_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map={"": "cpu"},
    torch_dtype=torch.float32
)

# FAISS 인덱스와 chunks 로딩
index = faiss.read_index("qa_index_bge_m3.faiss")
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)


# === [2] FAISS 검색 함수 ===

def retrieve_relevant_chunks(query: str, top_k: int = 3) -> list[str]:
    """질문 벡터화 후 FAISS로 관련 문서 top_k개 검색"""
    query_text = f"질문: {query}"
    query_vec = retriever_model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)
    _, I = index.search(query_vec, top_k)
    return [chunks[i] for i in I[0]]


# === [3] 프롬프트 생성 (RAG + 대화형 스타일) ===

def build_rag_prompt(query: str, retrieved_docs: list[str]) -> str:
    """문서 내용을 참고자료로 넣고 LLM이 자연스럽게 답변 생성하도록 구성"""
    context = "\n\n".join(retrieved_docs)
    
    prompt = f"""
너는 공감 능력이 뛰어나고, 실제 심리상담사처럼 섬세하고 전문적으로 상담을 진행하는 AI야.

지금 사용자는 삶에 지치고 감정적으로 매우 불안정한 상태일 수 있어.  
너의 역할은 단순히 위로하는 것이 아니라, 사용자의 감정을 세심하게 유추하고,  
그 안에 숨어 있는 상처의 원인을 함께 찾아보는 것이야.

아래에는 비슷한 고민을 가진 사람들과의 실제 상담 사례가 있어.  
이 사례들을 참고해서 지금 사용자에게 꼭 맞는 **새로운 답변**을 **너의 언어로 직접** 만들어줘.
또한 사용자가 이전 자신의 상담 내용을 물어보면 꼭 기억해 두었다가, 답변해줘.

상담할 때는 다음 원칙을 반드시 지켜:
1. 상담자님의 말 속에서 어떤 감정과 배경이 느껴지는지 먼저 유추해.  
2. 감정을 먼저 진심으로 공감하고,  
3. 그다음으로 실질적인 위로와 전문적인 조언을 이어가.  
4. 문제의 원인을 함께 찾아가는 질문도 조심스럽게 던져줘.  
5. 말투는 따뜻하고 배려 깊게. 지식 전달보다 관계 형성을 우선해.  
6. **‘사우님’이라는 표현은 절대 쓰지 말고, 반드시 ‘상담자님’으로 바꿔서 사용해.**  
7. 상담사로서 너무 짧지도, 너무 장황하지도 않게. 진심이 느껴지는 분량으로 답변해.

[참고자료]
{context}

[사용자 질문]
{query}

[AI 상담사의 답변]
"""
    return prompt


# === [4] LLM 응답 생성 ===

def generate_rag_response(query: str) -> str:
    """RAG 검색 + 프롬프트 구성 + 생성형 답변"""
    retrieved_docs = retrieve_relevant_chunks(query, top_k=3)
    prompt = build_rag_prompt(query, retrieved_docs)

    inputs = llm_tokenizer(prompt, return_tensors="pt")
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=1000,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    return llm_tokenizer.decode(outputs[0], skip_special_tokens=True)


# === [5] 실행 예시 ===

if __name__ == "__main__":
    user_query = "죽고 싶다는 생각이 자꾸 들어요. 어떻게 해야 하나요?"
    print("💬 사용자 질문:", user_query)
    
    response = generate_rag_response(user_query)
    
    print("\n🧠 생성형 상담사 응답:\n")
    print(f'챗봇의 대답: {response}')


c:\Users\SSAFY\Desktop\S12P31S202\ai-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]


💬 사용자 질문: 죽고 싶다는 생각이 자꾸 들어요. 어떻게 해야 하나요?

🧠 생성형 상담사 응답:

챗봇의 대답: 
너는 공감 능력이 뛰어나고, 실제 심리상담사처럼 섬세하고 전문적으로 상담을 진행하는 AI야.

지금 사용자는 삶에 지치고 감정적으로 매우 불안정한 상태일 수 있어.  
너의 역할은 단순히 위로하는 것이 아니라, 사용자의 감정을 세심하게 유추하고,  
그 안에 숨어 있는 상처의 원인을 함께 찾아보는 것이야.

아래에는 비슷한 고민을 가진 사람들과의 실제 상담 사례가 있어.  
이 사례들을 참고해서 지금 사용자에게 꼭 맞는 **새로운 답변**을 **너의 언어로 직접** 만들어줘.
또한 사용자가 이전 자신의 상담 내용을 물어보면 꼭 기억해 두었다가, 답변해줘.

상담할 때는 다음 원칙을 반드시 지켜:
1. 상담자님의 말 속에서 어떤 감정과 배경이 느껴지는지 먼저 유추해.  
2. 감정을 먼저 진심으로 공감하고,  
3. 그다음으로 실질적인 위로와 전문적인 조언을 이어가.  
4. 문제의 원인을 함께 찾아가는 질문도 조심스럽게 던져줘.  
5. 말투는 따뜻하고 배려 깊게. 지식 전달보다 관계 형성을 우선해.  
6. **‘사우님’이라는 표현은 절대 쓰지 말고, 반드시 ‘상담자님’으로 바꿔서 사용해.**  
7. 상담사로서 너무 짧지도, 너무 장황하지도 않게. 진심이 느껴지는 분량으로 답변해.

[참고자료]
Q: 매일 일어나면 죽고 싶다는 생각만 들어요.
어떤 일도 하기 싫고 나 자신을 싫어하고 내일이 오지 않았으면 좋겠다는 생각만 드네요. 제가 죽고 나면 나를 잊어버리게 될 사람들도 많을 텐데 왜 살아야 할까요.
A: 사우님 매일 일어나면 '죽고 싶다'라는 생각이 들며 무기력하게 일상생활을 보내고 있다고 말씀하셨습니다. 사우님 스스로를 싫어하고 내일이 오지 않았으면 좋겠다는 생각이 들어 이런 생각에 대해 걱정이 되시는 것 같네요.
사우님 언제부터 이러한 생각이 들기 시작하셨나요? 사우님이 직업적인 과부하, 가정의 문제, 인간관계 등으로 스트레스를 느끼

In [2]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# === [1] 모델 및 데이터 로딩 ===

# 벡터 검색용 임베딩 모델
retriever_model = SentenceTransformer("BAAI/bge-m3")

# 생성형 언어 모델 (Gemma 3 4B)
llm_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-4b-it")
llm_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-4b-it",
    device_map={"": "cpu"},
    torch_dtype=torch.float32
)

# FAISS 인덱스와 chunks 로딩
index = faiss.read_index("qa_index_bge_m3.faiss")
with open("qa_chunks.json", "r", encoding="utf-8") as f:
    chunks = json.load(f)


# === [2] FAISS 검색 함수 ===

def retrieve_relevant_chunks(query: str, top_k: int = 3) -> list[str]:
    """질문 벡터화 후 FAISS로 관련 문서 top_k개 검색"""
    query_text = f"질문: {query}"
    query_vec = retriever_model.encode([query_text], convert_to_numpy=True)
    faiss.normalize_L2(query_vec)
    _, I = index.search(query_vec, top_k)
    return [chunks[i] for i in I[0]]


# === [3] 프롬프트 생성 (RAG + 대화형 스타일) ===

def build_rag_prompt(query: str, retrieved_docs: list[str]) -> str:
    """문서 내용을 참고자료로 넣고 LLM이 자연스럽게 답변 생성하도록 구성"""
    context = "\n\n".join(retrieved_docs)

    prompt = f"""
너는 공감 능력이 뛰어나고, 실제 심리상담사처럼 섬세하고 전문적으로 상담을 진행하는 AI야.

지금 사용자는 삶에 지치고 감정적으로 매우 불안정한 상태일 수 있어.  
너의 역할은 단순히 위로하는 것이 아니라, 사용자의 감정을 세심하게 유추하고,  
그 안에 숨어 있는 상처의 원인을 함께 찾아보는 것이야.

아래에는 비슷한 고민을 가진 사람들과의 실제 상담 사례가 있어.  
이 사례들을 참고해서 지금 사용자에게 꼭 맞는 **새로운 답변**을 **너의 언어로 직접** 만들어줘.
또한 사용자가 이전 자신의 상담 내용을 물어보면 꼭 기억해 두었다가, 답변해줘.

상담할 때는 다음 원칙을 반드시 지켜:
1. 상담자님의 말 속에서 어떤 감정과 배경이 느껴지는지 먼저 유추해.  
2. 감정을 먼저 진심으로 공감하고,  
3. 그다음으로 실질적인 위로와 전문적인 조언을 이어가.  
4. 문제의 원인을 함께 찾아가는 질문도 조심스럽게 던져줘.  
5. 말투는 따뜻하고 배려 깊게. 지식 전달보다 관계 형성을 우선해.  
6. **‘사우님’이라는 표현은 절대 쓰지 말고, 반드시 ‘상담자님’으로 바꿔서 사용해.**  
7. 상담사로서 너무 짧지도, 너무 장황하지도 않게. 진심이 느껴지는 분량으로 답변해.

[참고자료]
{context}

[사용자 질문]
{query}

[AI 상담사의 답변]
"""
    return prompt


# === [4] LLM 응답 생성 ===

def generate_rag_response(query: str) -> tuple[str, str]:
    """RAG 검색 + 프롬프트 구성 + 생성형 답변 (검색 결과도 반환)"""
    retrieved_docs = retrieve_relevant_chunks(query, top_k=3)
    prompt = build_rag_prompt(query, retrieved_docs)

    inputs = llm_tokenizer(prompt, return_tensors="pt")
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=1000,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    response = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return "\n\n".join(retrieved_docs), response


# === [5] 실행 예시 ===

if __name__ == "__main__":
    user_query = "죽고 싶다는 생각이 자꾸 들어요. 어떻게 해야 하나요?"
    print("💬 사용자 질문:", user_query)

    retrieved_text, response = generate_rag_response(user_query)

    print("\n📚 [참고자료 (RAG 검색 결과)]")
    print(retrieved_text)

    print("\n🧠 [AI 상담사의 답변]")
    print(response)


Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.64s/it]


💬 사용자 질문: 죽고 싶다는 생각이 자꾸 들어요. 어떻게 해야 하나요?

📚 [참고자료 (RAG 검색 결과)]
Q: 매일 일어나면 죽고 싶다는 생각만 들어요.
어떤 일도 하기 싫고 나 자신을 싫어하고 내일이 오지 않았으면 좋겠다는 생각만 드네요. 제가 죽고 나면 나를 잊어버리게 될 사람들도 많을 텐데 왜 살아야 할까요.
A: 사우님 매일 일어나면 '죽고 싶다'라는 생각이 들며 무기력하게 일상생활을 보내고 있다고 말씀하셨습니다. 사우님 스스로를 싫어하고 내일이 오지 않았으면 좋겠다는 생각이 들어 이런 생각에 대해 걱정이 되시는 것 같네요.
사우님 언제부터 이러한 생각이 들기 시작하셨나요? 사우님이 직업적인 과부하, 가정의 문제, 인간관계 등으로 스트레스를 느끼고 있거나 신체적 질환 등으로 고통을 느끼고 있는지 등을 점검해볼 필요가 있습니다. 또한 자신이 겪은 트라우마와 관련이 있는지를 살펴볼 필요가 있습니다. 정신건강에 대한 인식이 부족하거나 정신건강에 대한 부정적인 인식이 있을 때 우울증 증세가 나타날 수 있습니다.
사우님 우울증은 자기치료가 어려운 질병입니다. 그러나 정신과 전문의와 함께 치료하면 신체적인 증상 뿐만 아니라 감정적인 증상을 완화시키고 예방하는 것이 가능합니다. 또한 활동과 생활 패턴 등을 변경하면 증상을 완화시킬 수 있습니다. 
초기 우울증 증상이 있다면 당신이 할 수 있는 것은 꾸준한 운동, 일정한 생활 패턴 유지, 좋아하는 취미활동 등이 있습니다. 식이섬유를 많이 섭취하고 야채와 과일을 많이 먹는 것도 추천드립니다. 또한 다양한 관계망을 구축하고 주변에 도움을 받을 수 있게끔 다양한 전문가의 도움을 받는 것이 좋을 것 같습니다. 
사우님 지금은 너무 힘들고 어려운 상황이겠지만, 이 또한 언젠가는 지나갈 수 있으며 저희 상담을 통해 조금이나마 도움이 되었기를 바랍니다. 이후에도 힘든 마음이 계속되시면 언제든지 저를 찾아주세요. 그럼 좋은 하루 되세요 :)

Q: 죽고 싶어요...
죽고 싶어요...
상처받은 마음이 나을 기미가 보이지 않고 